In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from keras.layers import Conv2DTranspose, MaxPooling2D, concatenate, Dropout, Conv2D
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input


from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import tensorflow as tf

In [ ]:
train_df = pd.read_csv("C:\\Users\\Admin\\Desktop\\AI\\ML\\Pro\\Data\\train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("C:\\Users\\Admin\\Desktop\\AI\\ML\\Pro\\Data\\depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

In [ ]:
train_df["images"] = [np.array(load_img("C:\\Users\\Admin\\Desktop\\AI\\ML\\Pro\\Data\\train\\images\\{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img("C:\\Users\\Admin\\Desktop\\AI\\ML\\Pro\\Data\\train\\masks\\{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
print(train_df)

## Calculating the salt coverage and salt coverage classes

In [ ]:
img_size_ori = 101
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

In [ ]:
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
print(train_df["images"][0])
print(train_df.columns)

In [ ]:
def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)

img_size_ori = 101
img_size_target = 128

ids_train, ids_valid, x_train, x_valid, y_train, y_valid, cov_train, cov_test, depth_train, depth_test = train_test_split(
    train_df.index.values,
    np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1), 
    train_df.coverage.values,
    train_df.z.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state=1337)
print(len(x_train[0]))
print(ids_train[0])

## Ploting coverage

In [ ]:
def row_from_imagge(image):
    #pass
    res = []
    for row in image:
        res += list(row.T[0])
    return res    
        
        
def data_prepare_for_ploting(x_train):
    res = []
    for image in x_train:
        res.append(row_from_imagge(image))
    return res    


data_for_ploting = pd.DataFrame(data_prepare_for_ploting(x_train))
data_for_ploting["y"] = cov_train


print(pd.DataFrame(data_prepare_for_ploting(x_train)))

print(x_train[0][0].T[0])

In [ ]:
tsne_data_for_ploting = pd.DataFrame(TSNE(n_components=2).fit_transform(data_for_ploting.values),columns=['x1', 'x2'])
tsne_data_for_ploting["y"] = cov_train
points1 = tsne_data_for_ploting.plot.scatter(x='x1', y='x2', c='y',colormap='viridis')

In [ ]:
tsne_data_for_ploting = pd.DataFrame(PCA(n_components=2).fit_transform(data_for_ploting.values),columns=['x1', 'x2'])
tsne_data_for_ploting["y"] = cov_train
points1 = tsne_data_for_ploting.plot.scatter(x='x1', y='x2', c='y',colormap='viridis')

## Model NN 

In [ ]:
import tensorflow as tf
from model import *


X = tf.placeholder(tf.float32, [None, 128, 128, 1])
Y = tf.placeholder(tf.float32, [None, 128, 128, 1])
lr = tf.placeholder(tf.float32)




import tensorflow as tf
from keras.layers import Conv2DTranspose
from model import *



def neural_network_model(input_layer, start_neurons=16):
    
    # 128 -> 64
    print(input_layer)
    conv1, weights = new_conv_layer(input_layer, 1, 3, start_neurons * 1, use_pooling=False)
    print(conv1)
    conv1, weights = new_conv_layer(conv1, 16, 3, start_neurons * 1, use_pooling=False)
    print(conv1)
    pool1 = max_poling(conv1)
    print(pool1)
    pool1 = tf.nn.dropout(pool1, 0.25)
    print(pool1)
    print("__________________________________________")
    # 64 -> 32    
    conv2, weights = new_conv_layer(pool1,16, 3, start_neurons * 2, use_pooling=False)
    print(conv2)
    conv2, weights = new_conv_layer(conv2, 32, 3, start_neurons * 2, use_pooling=False)
    print(conv2)
    pool2 = max_poling(conv2)
    print(pool2)
    pool2 = tf.nn.dropout(pool2, 0.5)
    print(pool2)
    print("__________________________________________")
    # 32 -> 16    
    conv3, weights = new_conv_layer(pool2, 32, 3, start_neurons * 4, use_pooling=False)
    print(conv3)
    conv3, weights = new_conv_layer(conv3, 64, 3, start_neurons * 4, use_pooling=False)
    print(conv3)
    pool3 = max_poling(conv3)
    print(pool3)
    pool3 = tf.nn.dropout(pool3, 0.5)
    print(pool3)
    print("__________________________________________")
    # 16 -> 8
    conv4, weights = new_conv_layer(pool3, 64, 3, start_neurons * 8, use_pooling=False)
    print(conv4)
    conv4, weights = new_conv_layer(conv4, 128, 3, start_neurons * 8, use_pooling=False)
    print(conv4)
    pool4 = max_poling(conv4)
    print(pool4)
    pool4 = tf.nn.dropout(pool4, 0.5)
    print(pool4)
    print("__________________________________________")
    # Middle
    conv_middle, weights = new_conv_layer(pool4, 128, 3, start_neurons * 16, use_pooling=False)
    print("conv_middle = ",conv_middle)
    conv_middle, weights = new_conv_layer(conv_middle, 256, 3, start_neurons * 16, use_pooling=False)
    print("conv_middle = ",conv_middle)
    print("__________________________________________")
    deconv4 = deconv2d(conv_middle, 3, 16, 128, 256, tf.nn.relu, "DeConv2d_4")
    print(deconv4)
    uconv4 = tf.concat([deconv4, conv4],-1)
    print(uconv4)
    uconv4 = tf.nn.dropout(uconv4,0.5)
    print(uconv4)
    uconv4, weights = new_conv_layer(uconv4, 256, 3, start_neurons * 8,use_pooling=False)
    print(uconv4)
    uconv4, weights = new_conv_layer(uconv4, 128, 3, start_neurons * 8,use_pooling=False)
    print(uconv4)
    print("__________________________________________")
    deconv3 =  deconv2d(uconv4, 3, 32, 64, 128, tf.nn.relu, "DeConv2d_3")
    print(deconv3) # (None, 32, 32, 64)
    uconv3 = tf.concat([deconv3, conv3],-1)
    print(uconv3)
    uconv3 = tf.nn.dropout(uconv3,0.5)
    print(uconv3)
    uconv3, weights = new_conv_layer(uconv3, 128, 3, start_neurons * 4,use_pooling=False)
    print(uconv3)
    uconv3, weights = new_conv_layer(uconv3, 64, 3, start_neurons * 4,use_pooling=False)
    print(uconv3)
    print("__________________________________________")
    deconv2 =  deconv2d(uconv3, 3, 64, 32, 64, tf.nn.relu, "DeConv2d_2") # (None, 64, 64, 32)
    print(deconv2)
    uconv2 = tf.concat([deconv2, conv2],-1)
    print(uconv2)
    uconv2 = tf.nn.dropout(uconv2,0.5)
    print(uconv2)
    uconv2, weights = new_conv_layer(uconv2, 64, 3, start_neurons * 2,use_pooling=False)
    print(uconv2)
    uconv2, weights = new_conv_layer(uconv2, 32, 3, start_neurons * 2,use_pooling=False)
    print(uconv2)
    print("__________________________________________")
    deconv1 =  deconv2d(uconv2, 3, 128, 16, 32, tf.nn.relu, "DeConv2d_1") # (None, 128, 128, 16)
    print(deconv1)
    uconv1 = tf.concat([deconv1, conv1], -1)
    print(uconv1)
    uconv1 =tf.nn.dropout(uconv1,0.5)
    print(uconv1)
    uconv1, weights = new_conv_layer(uconv1, 32, 3, start_neurons,use_pooling=False)
    print(uconv1)
    uconv1, weights = new_conv_layer(uconv1, 16, 3, start_neurons,use_pooling=False)
    print(uconv1)
    output_layer, weights = new_conv_layer(uconv1, 16, 1, 1,use_pooling=False)
    print(output_layer)
    print("__________________________________________")
    return output_layer 


    

logits = neural_network_model(X, start_neurons=16)

In [ ]:
x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)
learning_rate = 0.00001
training_epochs = 200
batch_size = 32


def get_next_batch(x,y,num_batches,indx):
    return np.split(x, num_batches)[indx], np.split(y, num_batches)[indx] 

# Define IoU metric
def mean_iou(y_true, y_pred):
    try:
        prec = []
        for t in np.arange(0.5, 1.0, 0.05):
            y_pred_ = tf.to_int32(y_pred > t)
            score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
            K.get_session().run(tf.local_variables_initializer())
            with tf.control_dependencies([up_opt]):
                score = tf.identity(score)
            prec.append(score)
    except Exception:
        print("mean_iou Error!")
    return K.mean(K.stack(prec), axis=0)
 

loss = mean_iou(Y, logits)
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [ ]:
# init
init = tf.global_variables_initializer()
#sess = tf.Session()
#sess.run(init)

def train_model(x_train, y_train, num_epoch, num_batches, learning_rate, cost_func):
    prediction = neural_network_model(x_train, 16)
    cost = cost_func(y_train, y_pred)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimaze(cost)
    
    with tf.Session as sess:
        sess.run(tf.initialize_all_veriables())
        
        for epoch in range(num_epoch):
            epoch_loss = 0
            for bath_ind in range(num_batches):
                x, y = get_next_batch(x_train, y_train, num_batches, indx)
                _, c = sess.run([optimizer, cost], feed_dict={x:x, y:y})
                epoch_loss += c
            print("Epoch: ", epoch, " completed ut of ", num_epoch," loss == ", epoch_loss)    